In [1]:
%store -r import_all_libs

In [2]:
import_all_libs

In [24]:
from datetime import date
import gc

In [4]:
fat_ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/faturamento_lojas_atuais.csv')
cen_exp = pd.read_csv('/home/tuts/datasets/uspvsufmg/cenarios_expansao.csv')
ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/lojas_atuais.csv')

In [31]:
ljs_atual.cod_loja.nunique()

3130

In [5]:
cen_exp.head()

cod_loja         cod_ap  cod_municipio           feature_01  \
0        10  2933307005008        2933307  FEATURE_01_VALUE_08   
1        13  2916401003002        2916401  FEATURE_01_VALUE_10   
2        16  2928703003003        2928703                  NaN   
3        21  2906006003001        2906006                  NaN   
4        28  3550308005058        3550308  FEATURE_01_VALUE_10   

            feature_02  feature_03  feature_04           feature_05  \
0  FEATURE_02_VALUE_02       False       50.00  FEATURE_05_VALUE_03   
1  FEATURE_02_VALUE_04       False       41.00  FEATURE_05_VALUE_03   
2  FEATURE_02_VALUE_04       False       42.12  FEATURE_05_VALUE_05   
3  FEATURE_02_VALUE_04       False       30.00  FEATURE_05_VALUE_03   
4  FEATURE_02_VALUE_02        True       44.31  FEATURE_05_VALUE_03   

   feature_06  feature_07  feature_08  feature_09  feature_10  \
0           6        0.13        0.13      116.19        2.91   
1           4        0.12        0.11       26.35        1.31   
2           2        0.10        0.09       35.36        1.97   
3           6        0.13        0.13        9.86        1.38   
4           8        0.19        0.18     5593.91        2.82   

            feature_11           feature_12  feature_13  feature_14  \
0  FEATURE_11_VALUE_01  FEATURE_12_VALUE_01      100.00       93.40   
1                  NaN                  NaN       84.30       77.50   
2                  NaN                  NaN         nan         nan   
3                  NaN                  NaN       85.00       76.90   
4  FEATURE_11_VALUE_02  FEATURE_12_VALUE_01         nan         nan   

   feature_15  feature_16  feature_17  feature_18  
0      100.00      100.00       71.70       88.80  
1      100.00      100.00       63.50       78.80  
2         nan         nan         nan         nan  
3      100.00      100.00      100.00       92.90  
4         nan         nan         nan         nan

In [6]:
df = fat_ljs_atual.merge(ljs_atual, on='cod_loja', how='inner')
df["datetime"] = pd.to_datetime(df.ano*100 + df.mes, format='%Y%m')
df.drop(["ano", "mes"], axis=1, inplace=True)

In [7]:
s = set(fat_ljs_atual.columns) - set(['ano', 'mes'])
s.add('datetime')
s

{'categoria', 'cod_loja', 'datetime', 'qtde', 'receita'}

In [8]:
fat_mes_df = df[list(s)].groupby(["cod_loja", "datetime"] , as_index=False).sum()

In [9]:
df_fat_mes = fat_mes_df.merge(ljs_atual, on="cod_loja", how='inner')

In [10]:
lojas_sem_24_meses = []
for i in df_fat_mes["cod_loja"].unique():
    if df_fat_mes[df_fat_mes["cod_loja"] == i].shape[0] != 24:
        lojas_sem_24_meses.append(i)

train_df = df_fat_mes[~df_fat_mes["cod_loja"].isin(lojas_sem_24_meses).values]

# Imputing

In [87]:
print('## FEATURE 1 ##')
print(train_df.feature_01.describe())
print('## FEATURE 11 ##')
print(train_df.feature_11.describe())
print('## FEATURE 12 ##')
print(train_df.feature_12.describe())
print('## FEATURE 13 ##')
print(train_df.feature_13.describe())
print('## FEATURE 14 ##')
print(train_df.feature_14.describe())
print('## FEATURE 15 ##')
print(train_df.feature_15.describe())
print('## FEATURE 16 ##')
print(train_df.feature_16.describe())
print('## FEATURE 17 ##')
print(train_df.feature_17.describe())
print('## FEATURE 18 ##')
print(train_df.feature_18.describe())

## FEATURE 1 ##
count                   26972
unique                      4
top       FEATURE_01_VALUE_10
freq                    15074
Name: feature_01, dtype: object
## FEATURE 11 ##
count                    5849
unique                      7
top       FEATURE_11_VALUE_02
freq                     2304
Name: feature_11, dtype: object
## FEATURE 12 ##
count                    5849
unique                      2
top       FEATURE_12_VALUE_01
freq                     3632
Name: feature_12, dtype: object
## FEATURE 13 ##
count   20949.00
mean       94.81
std         8.59
min        44.10
25%        92.20
50%       100.00
75%       100.00
max       100.00
Name: feature_13, dtype: float64
## FEATURE 14 ##
count   20949.00
mean       83.68
std        13.54
min         0.00
25%        79.70
50%        86.90
75%        91.90
max       100.00
Name: feature_14, dtype: float64
## FEATURE 15 ##
count   20949.00
mean       82.44
std        34.60
min         0.00
25%        93.30
50%       100.00
75%

In [11]:
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
has_na_cols

['feature_01',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18']

In [12]:
for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        print(train_df[na_col].mode()[0])
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)

FEATURE_01_VALUE_10
FEATURE_11_VALUE_02
FEATURE_12_VALUE_01



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
cols = train_df.columns

In [14]:
model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year

In [15]:
str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)

In [16]:
str_cols.append('datetime') 
str_cols.append('receita')
str_cols.append('qtde')

In [17]:
str_cols

['feature_01',
 'feature_02',
 'feature_05',
 'feature_11',
 'feature_12',
 'datetime',
 'receita',
 'qtde']

In [18]:
train = model_df[~model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]
test = model_df[model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]

X_train = train.drop(str_cols, axis=1)
y_train = train["receita"]

X_test = test.drop(str_cols, axis=1)
y_test = test["receita"]

In [19]:
X_train.columns

Index(['cod_loja', 'cod_ap', 'cod_municipio', 'feature_03', 'feature_04',
       'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'month', 'quarter', 'year'],
      dtype='object')

# Train

In [20]:
errors = list()

In [21]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)

rf.fit(X_train.values, y_train.values)
y_pred = rf.predict(X_test)
m = np.sqrt(mean_squared_error(y_test, y_pred))
errors.append(m)
m

34720.24976349914

In [43]:
lojas = ljs_atual.cod_loja.unique()

In [44]:
len(lojas)

3130

* todas as lojas:
    - valores da loja
    - datas tem que ser 1° trimestre de 2018
    - preve
    - coloca no df

In [26]:
del model_df, train, test, X_train, X_test, y_train, y_test, train_df
gc.collect()

181

# Submission

In [45]:
# copy df_fat_mes
train_df = df_fat_mes

print(train_df.cod_loja.nunique())

# get cols with nans
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
# input values
for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        print(train_df[na_col].mode()[0])
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)

# get cols for model
cols = train_df.columns
model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year

# get str and unused cols
str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)
str_cols.append('datetime') 
str_cols.append('receita')
str_cols.append('qtde')

# train test split
train = model_df[~model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]
test = model_df[model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]

print(model_df.cod_loja.nunique())

# X_train = train.drop(str_cols, axis=1)
# y_train = train["receita"]
# X_test = test.drop(str_cols, axis=1)
# y_test = test["receita"]

# predict
train_cols = X_train.columns.values.tolist()
preds = []
i = 0
for loja in lojas:
    Xloja = model_df[model_df.cod_loja == loja][:1]
    i += 1
    Xloja = Xloja[train_cols]
    Xloja = pd.DataFrame(Xloja.values.tolist()*3, columns=Xloja.columns)
    Xloja.month = pd.Series([1,2,3]).values
    Xloja.quarter = pd.Series([1,1,1]).values
    Xloja.year = pd.Series([2018,2018,2018]).values
    yhat = rf.predict(Xloja)
    preds.append((loja, yhat))
print(i)

3130
3130
3130


In [46]:
submission = pd.DataFrame({'cod_loja': [i[0] for i in preds], 'faturamento': [np.sum(i[1]) for i in preds]})
submission.cod_loja.nunique()

3130

In [47]:
submission.to_csv('/home/tuts/repos/others/usp-vs-ufmg/tutss/desafio_1.csv', index=False, sep=';')